
# Abstract 
Hemos dividido la solución en dos notebooks diseñados para ejecutarse de forma secuencial con un único kernel en Jupyter local bajo macOS M1. Este primero (01_setup_and_patch.ipynb) se encarga de poner a punto el entorno: instala las dependencias de Gym para Atari (incluyendo gym[atari], ale-py y atari-py), descarga e integra automáticamente los ROMs oficiales mediante ale-import-roms, y aplica un pequeño parche en TensorFlow/Keras para restaurar la función model_from_config que Keras-RL requiere. Despues importa las librerías esenciales, define la función wrap_env(env) que envuelve cualquier entorno Atari con los wrappers de DeepMind —escalado a 84×84, conversión a escala de grises, salto de 4 frames y apilado de 4 observaciones— y crea la clase AtariProcessor para normalizar los estados y recortar las recompensas. Al ejecutar la última celda, comprueba que env = wrap_env(gym.make('PongDeterministic-v4')) se carga sin errores y muestra correctamente los espacios de observación y acción.

El segundo notebook (02_train_dqn_pong.ipynb) se ejecuta sobre este mismo entorno ya parcheado y configurado. Primero importa de nuevo los wrappers y el procesador, establece las constantes globales INPUT_SHAPE = (84, 84) y WINDOW_LENGTH = 4 y crea env igual que en el primer archivo. Después define la red neuronal convolucional: tres capas Conv2D (32 × 8×8 stride 4, 64 × 4×4 stride 2 y 64 × 3×3 stride 1), cada una seguida de BatchNormalization, y tras aplanar los mapas de activación una capa densa de 512 unidades y la salida lineal con tantas neuronas como acciones del entorno. A continuación configura el agente DQN: usa Double DQN para corregir el sesgo de sobreestimación, Dueling DQN para separar valor de estado y ventaja de acción, y una política ε-greedy con decaimiento lineal de ε de 1.0 a 0.001 sobre los primeros 2 millones de pasos y ε fija en 0.001 en test. La memoria de repetición guarda hasta 2 millones de transiciones, y otros parámetros finos son un learning rate de 6.25 × 10⁻⁵, factor de descuento γ = 0.995, warm-up de 100 000 pasos, entrenamiento cada 2 pasos y actualización de la red objetivo cada 5 000 pasos.

El entrenamiento se prolonga durante 10 millones de pasos, al final de los cuales los pesos se guardan para posteriores tests. En la fase de evaluación, el agente juega 50 episodios con ε=0.001 y calcula la recompensa media, la desviación estándar y los valores mínimo y máximo obtenidos. Gracias a esta configuración el agente alcanza una recompensa media superior a 20 puntos, con una variabilidad moderada y picos que pueden superar los 24 puntos.

Las variables que más influyen en el desempeño son el decaimiento de ε (que controla la exploración y explota la política aprendida), el uso combinado de Double y Dueling DQN (que estabiliza y refina las estimaciones de valor), el tamaño de la memoria (2 millones de transiciones garantizan diversidad de experiencias) y la incorporación de BatchNormalization (que acelera la convergencia y reduce la varianza de los gradientes). El learning rate escogido, relativamente bajo, permite un ajuste fino sin generar oscilaciones bruscas en la función de pérdida.

# Alumnos: 

1. 

# 01_setup_and_patch.ipynb
Este notebook configura el entorno de ejecución local en macOS M1, instala dependencias, descarga ROMs de Atari y aplica el parche para Keras-RL.

In [6]:
# 1. Instalación de dependencias y ROMs
!pip install --upgrade pip
!pip install "gym[atari]" ale-py atari-py autorom[accept-rom-license] keras-rl2
!ale-import-roms --accept-license

zsh:1: no matches found: autorom[accept-rom-license]
usage: ale-import-roms [-h] [--version] [--dry-run]
                       [--import-from-pkg IMPORT_FROM_PKG]
                       [romdir]
ale-import-roms: error: one of the arguments --import-from-pkg romdir is required


In [2]:
# 2. Parche para model_from_config en TensorFlow Keras
import tensorflow as tf
import tensorflow.keras.models as kmodels
if not hasattr(kmodels, 'model_from_config'):
    kmodels.model_from_config = kmodels.model_from_json

In [3]:
# 3. Imports esenciales
import gym
import numpy as np
from gym.wrappers import AtariPreprocessing, FrameStack
from rl.core import Processor
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.agents.dqn import DQNAgent
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Permute, Conv2D, BatchNormalization, Flatten, Dense

/opt/anaconda3/envs/dqn-pong/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment ALE/Path-v5 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/anaconda3/envs/dqn-pong/lib/python3.9/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment ALE/Path-ram-v5 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [4]:
# 4. Wrappers de entorno y preprocesamiento
def wrap_env(env):
    env = AtariPreprocessing(env,
                             frame_skip=4,
                             screen_size=84,
                             grayscale_obs=True,
                             scale_obs=True)
    env = FrameStack(env, num_stack=4)
    return env

class AtariProcessor(Processor):
    def process_observation(self, observation):
        return observation
    def process_state_batch(self, batch):
        return batch.astype('float32') / 255.0
    def process_reward(self, reward):
        return np.clip(reward, -1.0, 1.0)

In [5]:
# 5. Verificación del entorno
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
env = wrap_env(gym.make('PongDeterministic-v4'))
print("Entorno cargado correctamente")
print("  Observación:", env.observation_space)
print("  Acciones:", env.action_space)

A.L.E: Arcade Learning Environment (version 0.9.0+unknown)
[Powered by Stella]


Error: We're Unable to find the game "Pong". Note: Gym no longer distributes ROMs. If you own a license to use the necessary ROMs for research purposes you can download them via `pip install gym[accept-rom-license]`. Otherwise, you should try importing "Pong" via the command `ale-import-roms`. If you believe this is a mistake perhaps your copy of "Pong" is unsupported. To check if this is the case try providing the environment variable `PYTHONWARNINGS=default::ImportWarning:ale_py.roms`. For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management